# Fancy but Useful Tensor Operations

In [26]:
import torch
import numpy as np
assert torch.cuda.is_available(), "CUDA is not available"

## 基础语法

### 创建操作

#### 和 NumPy 数组的转换

In [27]:
# 使用 `torch.from_numpy()` (共享内存)
a_array = np.array([1, 2, 3])
a_tensor = torch.from_numpy(a_array)
print("Original PyTorch tensor:", a_tensor)
a_array[0] = 10  # 修改 NumPy 数组
print("Tensor after modifying NumPy array:", a_tensor)

Original PyTorch tensor: tensor([1, 2, 3])
Tensor after modifying NumPy array: tensor([10,  2,  3])


In [28]:
# 使用 `torch.tensor()` (复制内存)
b_array = np.array([[1, 2], [3, 4]])
b_tensor = torch.tensor(b_array)
print("Original PyTorch tensor:")
print(b_tensor)
b_array[0, 0] = 10  # 修改 NumPy 数组
print("Tensor after modifying NumPy array:")
print(b_tensor)

Original PyTorch tensor:
tensor([[1, 2],
        [3, 4]])
Tensor after modifying NumPy array:
tensor([[1, 2],
        [3, 4]])


In [29]:
# 从 Tensor 转换为 NumPy 数组
c_tensor = torch.tensor([1, 2, 3], device='cuda')  # 创建一个在 GPU 上的 Tensor
try: # GPU 上的 Tensor 不能直接转换为 NumPy 数组
    c_array = c_tensor.numpy()
except Exception as e:
    print(e)
c_array = c_tensor.cpu().numpy()
print("NumPy array from CPU tensor:", c_array)

can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.
NumPy array from CPU tensor: [1 2 3]


#### 特殊 Tensor 构造

In [30]:
a = torch.rand(2, 3)
print("Random tensor a:")
print(a)

b = torch.zeros_like(a)
print("\nTensor b with same shape as a:")
print(b)

Random tensor a:
tensor([[0.0300, 0.8429, 0.9991],
        [0.9456, 0.4415, 0.2493]])

Tensor b with same shape as a:
tensor([[0., 0., 0.],
        [0., 0., 0.]])


In [31]:
a = torch.arange(1, 6, 2)
print("Tensor a:", a)

Tensor a: tensor([1, 3, 5])


### 基本属性

In [32]:
a = torch.zeros(2, 3, 4, device='cuda')
print("Rank of a:", a.dim())
print("Shape of a:", a.shape)
print("Shape of dim 1:", a.shape[1])
print("Datatype of a:", a.dtype)
print("Device of a:", a.device)

Rank of a: 3
Shape of a: torch.Size([2, 3, 4])
Shape of dim 1: 3
Datatype of a: torch.float32
Device of a: cuda:0


### 数据类型及设备

In [33]:
x0 = torch.zeros(2, 3, dtype=torch.float16, device='cpu')
print("x0.dtype:", x0.dtype)
print("x0.device:", x0.device)
x1 = x0.float()
print("x1.dtype:", x1.dtype)
x2 = x0.long()
print("x2.dtype:", x2.dtype)
x3 = x0.to(torch.int32)
print("x3.dtype:", x3.dtype)
x4 = x0.to("cuda")
print("x4.device:", x4.device)

x0.dtype: torch.float16
x0.device: cpu
x1.dtype: torch.float32
x2.dtype: torch.int64
x3.dtype: torch.int32
x4.device: cuda:0


## 索引操作

### 单元素索引

In [34]:
a = torch.tensor([[1, 2, 3], [4, 5, 6]])
print("a[0, 1]:", a[0, 1])
print("a[0, 1].item():", a[0, 1].item())

a[0, 1]: tensor(2)
a[0, 1].item(): 2


### 切片索引

In [35]:
a = torch.tensor([10, 11, 12, 13, 14, 15, 16])
print("a[2:5] ", a[2:5])  # Elements between index 2 and 5
print("a[:-1] ", a[:-1])  # All elements except the last one
print("a[::2] ", a[::2])  # Every second element
print("a[:] ", a[:])      # All elements

a[2:5]  tensor([12, 13, 14])
a[:-1]  tensor([10, 11, 12, 13, 14, 15])
a[::2]  tensor([10, 12, 14, 16])
a[:]  tensor([10, 11, 12, 13, 14, 15, 16])


In [36]:
b = torch.tensor(
    [[1, 2, 3, 4],
     [5, 6, 7, 8],
     [9, 10, 11, 12]]
)
# Single row
print("Single row:")
print(b[1, :], b[1, :].shape)  # Equivalent to b[1]
print(b[1:2, :], b[1:2, :].shape)
# Single column
print("\nSingle column:")
print(b[:, 2], b[:, 2].shape)
print(b[:, 2:3], b[:, 2:3].shape)
# All columns except the last one
print("\nAll columns except the last one:")
print(b[:, :-1], b[:, :-1].shape)

Single row:
tensor([5, 6, 7, 8]) torch.Size([4])
tensor([[5, 6, 7, 8]]) torch.Size([1, 4])

Single column:
tensor([ 3,  7, 11]) torch.Size([3])
tensor([[ 3],
        [ 7],
        [11]]) torch.Size([3, 1])

All columns except the last one:
tensor([[ 1,  2,  3],
        [ 5,  6,  7],
        [ 9, 10, 11]]) torch.Size([3, 3])


In [37]:
c = torch.zeros(2, 4, dtype=torch.int64)
c[:, :2] = 1
c[:, 2:] = torch.tensor([[2, 3], [4, 5]])
print(c)

tensor([[1, 1, 2, 3],
        [1, 1, 4, 5]])


### 整数 Tensor 索引

In [38]:
a = torch.arange(12).reshape(3, 4)
print("Original tensor a:")
print(a)

idx = torch.tensor([0, 0, 2, 1, 1])
print('\nReordered rows:')
print(a[idx])

idx = torch.tensor([3, 2, 1, 0])
print('\nReordered columns:')
print(a[:, idx])

Original tensor a:
tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])

Reordered rows:
tensor([[ 0,  1,  2,  3],
        [ 0,  1,  2,  3],
        [ 8,  9, 10, 11],
        [ 4,  5,  6,  7],
        [ 4,  5,  6,  7]])

Reordered columns:
tensor([[ 3,  2,  1,  0],
        [ 7,  6,  5,  4],
        [11, 10,  9,  8]])


In [39]:
b = torch.arange(1, 10).reshape(3, 3)
print("Original tensor b:")
print(b)

idx = torch.tensor([0, 1, 2])
print('\nGet the diagonal:')
print(b[idx, idx])

print('\nSet the diagonal to 0:')
b[idx, idx] = 0
print(b)

Original tensor b:
tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]])

Get the diagonal:
tensor([1, 5, 9])

Set the diagonal to 0:
tensor([[0, 2, 3],
        [4, 0, 6],
        [7, 8, 0]])


In [40]:
c = torch.arange(1, 13).reshape(4, 3)
print("Original tensor c:")
print(c)

idx0 = torch.arange(c.shape[0])
idx1 = torch.tensor([1, 2, 1, 0])
print('\nGet elements using index arrays:')
print(c[idx0, idx1])

Original tensor c:
tensor([[ 1,  2,  3],
        [ 4,  5,  6],
        [ 7,  8,  9],
        [10, 11, 12]])

Get elements using index arrays:
tensor([ 2,  6,  8, 10])


### 布尔 Tensor 索引

In [41]:
a = torch.rand(3, 4)
print("Original tensor:")
print(a)

mask = (a > 0.5)
print("\nMask tensor:")
print(mask)

print('\nSelecting elements with the mask:')
print(a[mask])

a[mask] = 0
print('\nAfter modifying with a mask:')
print(a)

Original tensor:
tensor([[0.6538, 0.2226, 0.1103, 0.4002],
        [0.5974, 0.4484, 0.4929, 0.8108],
        [0.1182, 0.5830, 0.1535, 0.8626]])

Mask tensor:
tensor([[ True, False, False, False],
        [ True, False, False,  True],
        [False,  True, False,  True]])

Selecting elements with the mask:
tensor([0.6538, 0.5974, 0.8108, 0.5830, 0.8626])

After modifying with a mask:
tensor([[0.0000, 0.2226, 0.1103, 0.4002],
        [0.0000, 0.4484, 0.4929, 0.0000],
        [0.1182, 0.0000, 0.1535, 0.0000]])


## 变形操作

### 改变逻辑形状

In [42]:
x0 = torch.arange(1, 9).reshape(2, 4)
print('Original tensor:')
print(x0)
print('shape:', x0.shape)

x1 = x0.view(-1)  # Equivalent to x1 = x0.flatten()
print('\nFlattened tensor:')
print(x1)
print('shape:', x1.shape)

x2 = x1.reshape(-1, 1)
print('\nColumn vector:')
print(x2)
print('shape:', x2.shape)

x3 = x1.view(2, 2, 2)
print('\nRank 3 tensor:')
print(x3)
print('shape:', x3.shape)

Original tensor:
tensor([[1, 2, 3, 4],
        [5, 6, 7, 8]])
shape: torch.Size([2, 4])

Flattened tensor:
tensor([1, 2, 3, 4, 5, 6, 7, 8])
shape: torch.Size([8])

Column vector:
tensor([[1],
        [2],
        [3],
        [4],
        [5],
        [6],
        [7],
        [8]])
shape: torch.Size([8, 1])

Rank 3 tensor:
tensor([[[1, 2],
         [3, 4]],

        [[5, 6],
         [7, 8]]])
shape: torch.Size([2, 2, 2])


In [43]:
a = torch.arange(12).reshape(3, 4)
b = a.t() # .t() makes the tensor non-contiguous
print("Is b contiguous?", b.is_contiguous())
print("\nOriginal tensor b:")
print(b)

try:
    c = b.view(2, 6)
except Exception as e:
    print("\nview() Failed:", e)

d = b.reshape(2, 6)
print("\nreshape() Succeeded:")
print(d)
print("\nIs d contiguous?", d.is_contiguous())

Is b contiguous? False

Original tensor b:
tensor([[ 0,  4,  8],
        [ 1,  5,  9],
        [ 2,  6, 10],
        [ 3,  7, 11]])

view() Failed: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.

reshape() Succeeded:
tensor([[ 0,  4,  8,  1,  5,  9],
        [ 2,  6, 10,  3,  7, 11]])

Is d contiguous? True


#### 改变维度顺序

In [49]:
x0 = torch.arange(1, 25).reshape(2, 3, 4)
print('Original tensor:')
print(x0)
print('shape:', x0.shape)

x1 = x0.transpose(0, 1)
print('\nSwap axes 0 and 1:')
print(x1)
print(x1.shape)

x2 = x0.permute(1, 2, 0)
print('\nPermute axes')
print(x2)
print('shape:', x2.shape)

Original tensor:
tensor([[[ 1,  2,  3,  4],
         [ 5,  6,  7,  8],
         [ 9, 10, 11, 12]],

        [[13, 14, 15, 16],
         [17, 18, 19, 20],
         [21, 22, 23, 24]]])
shape: torch.Size([2, 3, 4])

Swap axes 0 and 1:
tensor([[[ 1,  2,  3,  4],
         [13, 14, 15, 16]],

        [[ 5,  6,  7,  8],
         [17, 18, 19, 20]],

        [[ 9, 10, 11, 12],
         [21, 22, 23, 24]]])
torch.Size([3, 2, 4])

Permute axes
tensor([[[ 1, 13],
         [ 2, 14],
         [ 3, 15],
         [ 4, 16]],

        [[ 5, 17],
         [ 6, 18],
         [ 7, 19],
         [ 8, 20]],

        [[ 9, 21],
         [10, 22],
         [11, 23],
         [12, 24]]])
shape: torch.Size([3, 4, 2])


## 计算操作

### 归约操作

In [53]:
a = torch.arange(12).reshape(2, 3, 2)
print("Original tensor:")
print(a, a.shape)

print('\nSum over entire tensor:')
print(a.sum(), a.sum().shape)

print('\nSum over the first dimension:')
print(a.sum(dim=0), a.sum(dim=0).shape)

print('\nSum over the second dimension:')
print(a.sum(dim=1), a.sum(dim=1).shape)

print('\nSum over the last dimension:')
print(a.sum(dim=-1), a.sum(dim=-1).shape)


Original tensor:
tensor([[[ 0,  1],
         [ 2,  3],
         [ 4,  5]],

        [[ 6,  7],
         [ 8,  9],
         [10, 11]]]) torch.Size([2, 3, 2])

Sum over entire tensor:
tensor(66) torch.Size([])

Sum over the first dimension:
tensor([[ 6,  8],
        [10, 12],
        [14, 16]]) torch.Size([3, 2])

Sum over the second dimension:
tensor([[ 6,  9],
        [24, 27]]) torch.Size([2, 2])

Sum over the last dimension:
tensor([[ 1,  5,  9],
        [13, 17, 21]]) torch.Size([2, 3])
